# 5. Generating cube map from intensity maps
In this section, we generate a cube map of Milky Way from the intensity maps.

## Load intensity maps

First, enter and run the following codes to import all modules used in this section, to define the folders for input and output, and to define the names of the planes. Here, we take $N=512$ as the number of cells in one direction.
```
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

mpl.rcParams['figure.autolayout'] = 'false'

src_dir = './intensity_maps'
dest_dir = './cube_map'

plane_name = ('posx', 'negx', 'posy', 'negy', 'posz', 'negz')

N = 512
```

Then, load all the intensity maps generated in the previous section. Enter the following code in the cell below and run it. The intensity maps are loaded in a `NumPy` array '`imap`'.

```
# make sure the directory 'dest_dir' is avairable
if not os.path.isdir(dest_dir):
    os.mkdir(dest_dir)

imap = np.empty((6,N,N), 'float32')

for ip in range(6):
    fni = '{0}/intensity_map_{1}_{2}.dat'.format(src_dir, N, plane_name[ip]) # Generate input filename
    data = np.fromfile(fni, 'float32') # Load the intensity map from file
    data.shape = (N,N) # reshape
    imap[ip] = data
    print('Loaded ' + "'" + fni + "'")
```

## Make image file

Let's visualize the intensity map with the PIL module. Enter the following code in the cell below and run it. In this code first allocates an array for images. Then, it defines a function to visualize the intensity maps as an image for each plane whose index is given by i1. As already mentioned in the previous section, we have to choose a factor to adjust image brightness. It is given by the argument of the function `fac`. 
```
# Allocate an array for images
img = [None]*6
for ip in range(6):
    img[ip] = Image.new('RGB', (N,N), (0,0,0))

# Define a function to render the intensity map
def render_intensity_map(ip, fac):
    for iy in range(N):
        for ix in range(N):
            val = fac*imap[ip][iy][ix]
            if val > 1:
                val = 1

            val = int(256*val)
            r = g = b = val
            iyr = N - iy -1 # reverse y coord
            img[ip].putpixel((ix,iyr), (r,g,b))
```

Then, let's render the intensity maps for all planes. Enter and run the following code in the cell below. Here, we choose the factor `fac = 500`.
```
# Render the intensity maps to images
fac = 500
for ip in range(6):
    render_intensity_map(ip,fac)
```

Then, define a function for preview the rendered cube map. Enter and run the following code to preview. If the displayed images are too bright or too dark, edit the value of the factor `fac` in the cell above and run it and then run the cell below again. If you want to eliminate the plane names in the preview, comment out the line of `plt.text`.

```
def preview_cube_map():
    plt.clf()
    plt.figure(figsize=(8,6))
    xt = yt = N/2
    
    fig_pos = (7, 5, 2, 10, 8, 6) # Position of subplot
    for ip in range(6):
        plt.subplot(3,4,fig_pos[ip])
        plt.axis('off') # removing axes
        plt.imshow(np.array(img[ip]))
        plt.text(xt,yt, plane_name[ip], ha='center', va='center', size=15, color='white')
    
    plt.subplots_adjust(0,0,1,1,0,0)
    plt.show()


preview_cube_map()
```

## Gamma correction for sRGB
Thus, we finally generated a Milky Way cube map from the Gaia's data. However, there is one more thing to do -- Gamma correction. Since the input and the output of display devices are generally not related with linearly, in order to reproduce the true brightness gradation in the display devices, corrections are needed.
The correction to be adopted depends on the kind of device. Here we make the correction for the sRGB color space. The correction from the linear input values to the output values for the sRGB is given by
$$
C_\mathrm{sRGB}
= 
\left\{
    \begin{array}{ll}
      12.92\ C_\mathrm{linear} & \textrm{for} \ C_\mathrm{linear} \le 0.0031308\\
      1.055\ C_\mathrm{linear}^{1/2.4} - 0.055 & \textrm{for} \ C_\mathrm{linear} > 0.0031308.
    \end{array}
\right.
$$
Enter the following code and run it to define a function for this correction.
```
def correct_gamma_0(lv):
    if (lv <= 0.0031308):
        lv = 12.92 * lv
    else:
        lv = 1.055 *  (lv**(1.0/2.4)) - 0.055
    return lv

# convert to NumPy universal function
correct_gamma = np.frompyfunc(correct_gamma_0, 1, 1)
```

Let's plot this function with matplotlib module. Enter the following code and run it.
```
# Graph
x = np.arange(0,1,0.01)
y = correct_gamma(x)
plt.figure(1)
plt.plot(x,y)
plt.title('Correction function for sRGB')
plt.xlabel('Linear value')
plt.ylabel('Corrected value')
plt.show()
```

Next, let's define a rendering function including this correction. Enter and run the following code to define it and generate images again with the correction. Here, we choose `fac=200`.
```
def render_intensity_map_with_correction(ip, fac):
    for iy in range(N):
        for ix in range(N):
            val = fac*imap[ip][iy][ix]
            if val > 1:
                val = 1

            val = correct_gamma(val)
            val = int(256*val)
            r = g = b = val
            iyr = N - iy -1 # reverse y coord
            img[ip].putpixel((ix,iyr), (r,g,b))

# Regenerate the images with the correction
fac = 200
for ip in range(6):
    render_intensity_map_with_correction(ip, fac)
```

Preview the corrected cube map by running the following code. If the image is too bright or too dark, edit the value of `fac` in the above cell and run it. Then, preview again.
```
preview_cube_map()
```

If you have generated the cube map with proper brightness, let's save the images into files. Enter and run the following code to save images into the folder '`cube_map`'.
```
#Save the images
for ip in range(6):
    fno = '{0}/milkyway_map_{1}_{2}.png'.format(dest_dir, N, plane_name[ip]) # Generate output filename
    img[ip].save(fno)
```

Congratulations!  
You have finally generated a Milky Way cube map from the Gaia's DR1 data by yourself!